---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [8]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [180]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    uni=pd.read_table('university_towns.txt', header=None)
    State=[]
    Town=[]
    uni.replace(to_replace='\([^)]*\)',value="",inplace=True,regex=True)
    uni.replace(to_replace='\[[^)]*\]',value="",inplace=True,regex=True)
    currentState=None
    for u in uni[0]:
        if u in list(states.values()):
            currentState=u
        else:
            State.append(currentState)
            Town.append(u)
    univ=pd.DataFrame({"State":State,'RegionName':Town})
    univ['University']=1
    ans=univ.set_index(['State','RegionName'])
    
    return ans
get_list_of_university_towns()


University
State         RegionName                               
Alabama       Auburn                                  1
              Florence                                1
              Jacksonville                            1
              Livingston                              1
              Montevallo                              1
              Troy                                    1
              Tuscaloosa                              1
              Tuskegee                                1
Alaska        Fairbanks                               1
Arizona       Flagstaff                               1
              Tempe                                   1
              Tucson                                  1
Arkansas      Arkadelphia                             1
              Conway                                  1
              Fayetteville                            1
              Jonesboro                               1
              Magnolia                                1
              Monticello                              1
              Russellville                            1
              Searcy                                  1
California    Angwin                                  1
              Arcata                                  1
              Berkeley                                1
              Chico                                   1
              Claremont                               1
              Cotati                                  1
              Davis                                   1
              Irvine                                  1
              Isla Vista                              1
              University Park, Los Angeles            1
...                                                 ...
Virginia      Wise                                    1
              Chesapeake                              1
Washington    Bellingham                              1
              Cheney                                  1
              Ellensburg                              1
              Pullman                                 1
              University District, Seattle            1
West Virginia Athens                                  1
              Buckhannon                              1
              Fairmont                                1
              Glenville                               1
              Huntington                              1
              Montgomery                              1
              Morgantown                              1
              Shepherdstown                           1
              West Liberty                            1
Wisconsin     Appleton                                1
              Eau Claire                              1
              Green Bay                               1
              La Crosse                               1
              Madison                                 1
              Menomonie                               1
              Milwaukee                               1
              Oshkosh                                 1
              Platteville                             1
              River Falls                             1
              Stevens Point                           1
              Waukesha                                1
              Whitewater                              1
Wyoming       Laramie                                 1

[517 rows x 1 columns]

In [174]:
def get_recession_start():
    gdp=pd.read_excel("gdplev.xls",skiprows=[0,1,2,3,4])
    gdp.columns=['year','current','Gdpy','drop','Quarter','drop2','Gdp',
                                                              'Gdpx']
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    short=gdp.ix[2:,['Quarter','Gdp']]
    Diff0=[0]+[(short.ix[i,'Gdp']-short.ix[i-1,'Gdp']) for i in range(3,len(gdp))]
    Diff1=[(short.ix[i+1,'Gdp']-short.ix[i,'Gdp']) for i in range(2,len(gdp)-1)]+[0]
    Diff2=[(short.ix[i+1,'Gdp']-short.ix[i,'Gdp']) for i in range(3,len(gdp)-1)]+[0,0]
    short['D1']=Diff1
    short['D2']=Diff2
    short['D0']=Diff0
    ress=short.loc[(short['D1']<0) & (short['D2']<0)&(short['D0']>=0)] #####here
    return ress['Quarter']
get_recession_start()


2      1947q1
9      1948q4
27     1953q2
44     1957q3
92     1969q3
111    1974q2
134    1980q1
140    1981q3
176    1990q3
247    2008q2
Name: Quarter, dtype: object

In [97]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp=pd.read_excel("gdplev.xls",skiprows=[0,1,2,3,4])
    gdp.columns=['year','current','Gdpy','drop','Quarter','drop2','Gdp',
                                                              'Gdpx']
    short=gdp.ix[2:,['Quarter','Gdp']]
    Diff1=[0]+[(short.ix[i,'Gdp']-short.ix[i-1,'Gdp']) for i in range(3,len(gdp))]
    Diff2=[0,0]+[(short.ix[i-1,'Gdp']-short.ix[i-2,'Gdp']) for i in range(4,len(gdp))]
    Diff0=[(short.ix[i+1,'Gdp']-short.ix[i,'Gdp']) for i in range(2,len(gdp)-1)]+[0]
    short['D1']=Diff1
    short['D2']=Diff2
    short['D0']=Diff0
    ress=short.loc[(short['D1']<0) & (short['D2']<0)&(short['D0']>=0)] #####here
    return ress['Quarter']
get_recession_end()

4      1947q3
11     1949q2
30     1954q1
46     1958q1
94     1970q1
114    1975q1
136    1980q3
142    1982q1
178    1991q1
251    2009q2
Name: Quarter, dtype: object

In [114]:
def get_recession_bottom():
    gdp=pd.read_excel("gdplev.xls",skiprows=[0,1,2,3,4])
    gdp.columns=['year','current','Gdpy','drop','Quarter','drop2','Gdp',
                                                            'Gdpx']
    short=gdp.ix[2:,['Quarter','Gdp']]
    ends=get_recession_end()
    bottom=[]    
    for i in range(3,len(gdp['Quarter'])-1):  
        if np.any(ends.isin([gdp.ix[i+1,'Quarter']])):
            bottom.append(gdp.ix[i,'Quarter'])
            
        
                   
        
    
    return bottom
get_recession_bottom()


['1947q2',
 '1949q1',
 '1953q4',
 '1957q4',
 '1969q4',
 '1974q4',
 '1980q2',
 '1981q4',
 '1990q4',
 '2009q1']

In [182]:
def convert_housing_data_to_quarters():
    dat=pd.read_csv("City_Zhvi_AllHomes.csv")
    years=[str(i) for i in range(1996,2017)]
    q1=['-01','-02','-03']
    q2=['-04','-05','-06']
    q3=['-07','-08','-09']
    q4=['-10','-11','-12']
   
    for y in years:
        for q in range(1,5):
            
          
            if q==1:
                s=q1
            elif q==2:
                s=q2
            elif q==3:
                s=q3
            elif q==4:
                s=q4
            else:
                print('error')
            
            
            if (y+s[0] in dat.columns)&(y+s[1] in dat.columns)&(y+s[2] in dat.columns):
                
                dat[y+'q'+str(q)]=np.mean(dat[[y+s[0],y+s[1],y+s[2]]],axis=1)
                del dat[y+s[0]]
                del dat[y+s[1]]
                del dat[y+s[2]]
            if (y+s[0] in dat.columns)&(y+s[1] in dat.columns):
                dat[y+'q'+str(q)]=np.mean(dat[[y+s[0],y+s[1]]],axis=1)
                del dat[y+s[0]]
                del dat[y+s[1]]
    dat.replace(to_replace={'State':states},inplace=True)
    newDat=dat.set_index(['State','RegionName'])           
    #newDat=dat.set_index('RegionName')           
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return newDat.loc[:,'2000q1':'2016q3']
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [184]:
def run_ttest():
    rStart=get_recession_start()
    rBottom=get_recession_bottom()
    bigHouse=convert_housing_data_to_quarters()
    bigHouse['Change']=bigHouse['2009q1']-bigHouse['2008q2']
    univ=get_list_of_university_towns()

    house =bigHouse.merge(univ,how='outer',left_index=True,right_index=True,indicator=True)
    
    #I don't see any university towns within the data provided.  I suppose I could find my own,
    # but I don't feel like I need t-test practice
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return house#univ
run_ttest()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Auburn                     NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
          Buhl              90800.000000   94600.000000   96500.000000   
          Calera           108933.333333  110366.666667  108000.000000   
          Center Point      80966.666667   81233.333333   81500.000000   
          Centreville       95300.000000   96566.666667   98000.000000   
          Chalkville        94100.000000   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          162066.666667  167033.333333  166900.000000   
          Chickasaw         51200.000000   53666.666667   54933.333333   
          Chunchula         80266.666667   81766.666667   82200.000000   
          Citronelle        64833.333333   66633.333333   68066.666667   
          Clay             120900.000000  122266.666667  123966.666667   
          Coden             62600.000000   64800.000000   66866.666667   
          Coker            118100.000000  120766.666667  118166.666667   
          Concord           78600.000000   78700.000000   80133.333333   
...                                  ...            ...            ...   
Wisconsin Wales                      NaN            NaN            NaN   
          Waterford        121200.000000  119433.333333  120200.000000   
          Waukesha         141266.666667  138433.333333  136733.333333   
          Waukesha                   NaN            NaN            NaN   
          Waunakee         187400.000000  191433.333333  192666.666667   
          Waupun            84000.000000   84000.000000   84400.000000   
          Wausau            69566.666667   69466.666667   71033.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        80933.333333   81933.333333   84066.666667   
          Weston            80166.666667   82700.000000   84166.666667   
          Wheatland        129800.000000  132133.333333  134933.333333   
          Whitelaw          96033.333333  101433.333333  108033.333333   
          Whitewater                 NaN            NaN            NaN   
          Williams Bay     122900.000000  115300.000000  110766.666667   
          Wilson           129033.333333  129366.666667  132433.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
          Woodruff                   NaN            NaN            NaN   
          Woodville                  N